In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm

# cr_cp = pm.df('cr_cp')
# fe_cp = pm.df('fe_cp')
df_jo = pm.df('df_jo')
df_jo = pm.sort("df_jo", ['created_at','created_at_fe']).reset_index()
df_jo = df_jo.drop(columns=['index'])

pd.options.display.max_columns = None

fields = ['id_cr','user_id','created_at','moderated_at','to_reimbur','transfer_type','stat_cr' ,'amount','fee',
          'n_fees','n_backs','needs_m_check_recov', 'n_recovery','n_inc_back','n_inc_fees',
          'n_cr_fe_w','n_cr_fe_m',  #'created_at_dow',
          'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment','type','recovery_status','created_at_slot' 
          # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
          ]


In [6]:
df = df_jo.copy()
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.sort_values(by=['user_id', 'created_at'])

# obtenemos la primera fila de cada 'user_id' (la más antigua)
primer_registro = df.groupby('user_id').first()
primer_registro['no_moderat'] = primer_registro['moderated_at'].isnull()
no_moderats = primer_registro.reset_index() #level='user_id')
no_moderats = primer_registro[primer_registro['no_moderat'] == True]
#resultat = no_moderats.reset_index()[['id_cr','id_fe','user_id','created_at','moderated_at','','no_moderat','transfer_type']]
resultat = no_moderats[no_moderats.transfer_type.isin(['regular','instant'])].reset_index()

#display(resultat.sort_values(['created_at']))
#display(resultat[resultat.user_id > 0 ][fields].sort_values(['created_at','created_at_fe']).reset_index(drop=True))
#display(resultat[fields].sort_values('created_at').head(4))
print(f"Usuarios nuevos: {resultat.user_id.count()}")

df['moderada'] = df['moderated_at'].apply(lambda x: 1 if not pd.isnull(x) else 0)
cnt = df[df['moderada'] == 1]
print(f"moderados: {cnt.moderada.count()}")

cnt = df[df['moderada'] == 0]
print(f"no moderados: {cnt.moderada.count()}")

display(df[df['user_id'] == 47].sort_values('created_at')[fields].reset_index())

Usuarios nuevos: 5027
moderados: 21757
no moderados: 10335


,index,id_cr,user_id,created_at,moderated_at,to_reimbur,transfer_type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check_recov,n_recovery,n_inc_back,n_inc_fees,n_cr_fe_w,n_cr_fe_m,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,type,recovery_status,created_at_slot
0,0,3,47,2019-11-19 13:57:53.511561,2019-11-20 12:16:50.000000,16 days 09:02:06.488439,regular,canceled,1.0,0.0,0,0,1,0,1,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2019-12-05 23:00:00.000000,16 days 09:02:06.488439,NaT,NaT,NaN,0,nice,13
1,53,118,47,2019-12-12 14:14:14.131479,2019-12-12 14:41:39.485394,6 days 23:59:59.999546,regular,rejected,100.0,0.0,0,0,1,0,2,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2019-12-19 14:14:14.131025,6 days 23:59:59.999546,NaT,NaT,NaN,0,nice,14
2,1143,1218,47,2020-04-16 08:42:15.429083,2020-04-16 09:15:24.585050,14 days 00:00:00.577917,regular,money_back,100.0,0.0,0,1,0,0,2,0,1,1,cr_regular,0,NaT,NaT,NaN,2020-05-08 21:35:33.858794,2020-04-30 08:42:16.007000,14 days 00:00:00.577917,NaT,NaT,NaN,0,nice,8
3,1771,1832,47,2020-05-15 16:04:10.449840,2020-05-15 16:08:34.000000,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,1,1,1,cancelled,72,2020-05-29 14:27:56.118917,2020-12-22 17:19:26.530567,Postpone Cash Request 1832,2020-08-04 18:19:04.558434,2020-09-17 22:00:00.000000,125 days 05:55:49.550160,2020-07-19 22:00:00,2020-08-03 22:00:00,before,postpone,nice,16
4,1772,1832,47,2020-05-15 16:04:10.449840,2020-05-15 16:08:34.000000,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,2,1,1,cancelled,71,2020-05-29 14:27:56.118917,2020-12-22 17:19:34.028812,Postpone Cash Request 1832,2020-08-04 18:19:04.558434,2020-09-17 22:00:00.000000,125 days 05:55:49.550160,2020-06-19 22:00:00,2020-07-19 22:00:00,before,postpone,nice,16
5,1773,1832,47,2020-05-15 16:04:10.449840,2020-05-15 16:08:34.000000,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,3,1,1,cancelled,47,2020-06-01 14:24:14.624776,2020-12-22 17:19:20.056527,Postpone Cash Request 1832,2020-08-04 18:19:04.558434,2020-09-17 22:00:00.000000,125 days 05:55:49.550160,2020-08-03 22:00:00,2020-08-18 22:00:00,before,postpone,nice,16
6,1774,1832,47,2020-05-15 16:04:10.449840,2020-05-15 16:08:34.000000,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,4,1,1,cancelled,21,2020-06-23 08:20:48.513423,2020-12-22 17:19:13.611324,Postpone Cash Request 1832,2020-08-04 18:19:04.558434,2020-09-17 22:00:00.000000,125 days 05:55:49.550160,2020-06-05 22:00:00,2020-06-19 22:00:00,before,postpone,nice,16
7,1775,1832,47,2020-05-15 16:04:10.449840,2020-05-15 16:08:34.000000,125 days 05:55:49.550160,regular,money_back,10.0,5.0,1,2,0,0,2,4,1,1,accepted,1367,2020-07-14 14:59:45.303324,2020-10-13 14:25:02.353414,Postpone Cash Request 1832,2020-08-04 18:19:04.558434,2020-09-17 22:00:00.000000,125 days 05:55:49.550160,2020-08-18 22:00:00,2020-09-17 22:00:00,before,postpone,nice,16
8,11530,10942,47,2020-08-05 09:34:16.818322,2020-08-05 09:35:50.000000,55 days 12:25:43.181678,instant,money_back,10.0,5.0,2,3,0,0,2,4,2,2,accepted,2792,2020-08-05 09:35:51.945346,2021-01-21 15:50:35.089962,Instant Payment Cash Request 10942,2020-09-22 15:05:36.000000,2020-09-29 22:00:00.000000,55 days 12:25:43.181678,NaT,NaT,after,instant_payment,nice,9
9,11531,10942,47,2020-08-05 09:34:16.818322,2020-08-05 09:35:50.000000,55 days 12:25:43.181678,instant,money_back,10.0,5.0,3,3,0,0,2,4,2,2,accepted,5609,2020-09-01 10:54:11.067922,2020-10-13 14:25:08.138501,Postpone Cash Request 10942,2020-09-22 15:05:36.000000,2020-09-29 22:00:00.000000,55 days 12:25:43.181678,2020-09-04 22:00:00,2020-09-29 22:00:00,before,postpone,nice,9


In [7]:
#df_jo = df_jo.drop(columns=['Mes_created_at'])
#df_jo_cp = df_jo.copy()
#df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
#df_jo.info()

#display(df_jo)

df = df_jo.copy()
pm.format_to_dates(df, time_format='s') # 'min','s'

pm.format_to_dates(df, time_format='d') # 'min','s'
df = df.sort_values(by=['user_id', 'created_at'])

# 34, 35 47 83 163 191 204 213
display(df[df['user_id'] == 47][fields].reset_index()) #47 tenemos un problema con n_fees
#display(df[df['n_backs'] <0][fields].reset_index().head(50)) #47 tenemos un problema con n_fees


,index,id_cr,user_id,created_at,moderated_at,to_reimbur,transfer_type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check_recov,n_recovery,n_inc_back,n_inc_fees,n_cr_fe_w,n_cr_fe_m,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,type,recovery_status,created_at_slot
0,0,3,47,2019-11-19,2019-11-20,16 days 09:02:06.488439,regular,canceled,1.0,0.0,0,0,1,0,1,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2019-12-05,16 days 09:02:06.488439,NaT,NaT,NaN,0,nice,13
1,53,118,47,2019-12-12,2019-12-12,6 days 23:59:59.999546,regular,rejected,100.0,0.0,0,0,1,0,2,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2019-12-19,6 days 23:59:59.999546,NaT,NaT,NaN,0,nice,14
2,1143,1218,47,2020-04-16,2020-04-16,14 days 00:00:00.577917,regular,money_back,100.0,0.0,0,1,0,0,2,0,1,1,cr_regular,0,NaT,NaT,NaN,2020-05-08,2020-04-30,14 days 00:00:00.577917,NaT,NaT,NaN,0,nice,8
3,1771,1832,47,2020-05-15,2020-05-15,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,1,1,1,cancelled,72,2020-05-29,2020-12-22,Postpone Cash Request 1832,2020-08-04,2020-09-17,125 days 05:55:49.550160,2020-07-19,2020-08-03,before,postpone,nice,16
4,1772,1832,47,2020-05-15,2020-05-15,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,2,1,1,cancelled,71,2020-05-29,2020-12-22,Postpone Cash Request 1832,2020-08-04,2020-09-17,125 days 05:55:49.550160,2020-06-19,2020-07-19,before,postpone,nice,16
5,1773,1832,47,2020-05-15,2020-05-15,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,3,1,1,cancelled,47,2020-06-01,2020-12-22,Postpone Cash Request 1832,2020-08-04,2020-09-17,125 days 05:55:49.550160,2020-08-03,2020-08-18,before,postpone,nice,16
6,1774,1832,47,2020-05-15,2020-05-15,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,4,1,1,cancelled,21,2020-06-23,2020-12-22,Postpone Cash Request 1832,2020-08-04,2020-09-17,125 days 05:55:49.550160,2020-06-05,2020-06-19,before,postpone,nice,16
7,1775,1832,47,2020-05-15,2020-05-15,125 days 05:55:49.550160,regular,money_back,10.0,5.0,1,2,0,0,2,4,1,1,accepted,1367,2020-07-14,2020-10-13,Postpone Cash Request 1832,2020-08-04,2020-09-17,125 days 05:55:49.550160,2020-08-18,2020-09-17,before,postpone,nice,16
8,11530,10942,47,2020-08-05,2020-08-05,55 days 12:25:43.181678,instant,money_back,10.0,5.0,2,3,0,0,2,4,2,2,accepted,2792,2020-08-05,2021-01-21,Instant Payment Cash Request 10942,2020-09-22,2020-09-29,55 days 12:25:43.181678,NaT,NaT,after,instant_payment,nice,9
9,11531,10942,47,2020-08-05,2020-08-05,55 days 12:25:43.181678,instant,money_back,10.0,5.0,3,3,0,0,2,4,2,2,accepted,5609,2020-09-01,2020-10-13,Postpone Cash Request 10942,2020-09-22,2020-09-29,55 days 12:25:43.181678,2020-09-04,2020-09-29,before,postpone,nice,9


In [8]:
pm.format_to_dates(df, time_format='d') # 'min','s'
df = df.sort_values(by=['user_id', 'created_at'])
# 2024-12-11 TODO Alejandro Casos en los que poner n_backs -1 nos da un resultado incorrecto:
# 34, 35 47 83 163 191 204 213

# # # # Para stat_cr == "money_back" & stat_fe == "accepted" acumulamos el numero de operaciones con feeds
# # df = df.sort_values(['created_at','created_at_fe'])
# money_back = df['stat_cr'] == "money_back"
# fee_accepted = df['stat_fe'] == "accepted"

# # df['n_fees'] = money_back & fee_accepted & (df['fee'] > 0)
# # df['n_fees'] = df.groupby('user_id')['n_fees'].cumsum()

# # # # Para stat_cr == "money_back" & stat_fe == "accepted" acumulamos el numero de operaciones de tipo money_back        
# # df = df.sort_values(['created_at','created_at_fe'])
# unique_cr = money_back & (df['amount'] > 0) & (~df.duplicated(subset=['id_cr'], keep='first'))
# df['n_backs'] = unique_cr.groupby(df['user_id']).cumsum() # -1 # 2024-12-11 Cesc no podemos hacer el -1 esto nos deja casos en negativo que son claramente erroneos

# # # Para stat_cr != good_cr | stat_fe != good_fe acumulamos el numero de operaciones de tipo money_back
# # good_cr = ['approved', 'money_sent', 'pending', 'direct_debit_sent', 'active', 'money_back']
# # good_fe = ['confirmed', 'accepted', 'cr_regular']
# # is_good_cr = df['stat_cr'].isin(good_cr)
# # df = df.sort_values(['created_at','created_at_fe'])
# # is_good_fe = df['stat_fe'].isin(good_fe)
# # bad_recovery_status_fe = df['recovery_status'] != "nice"
# # df['n_incidents'] = ( (~is_good_cr) | (~is_good_fe) | (bad_recovery_status_fe)  ) & (df['amount'] > 0)
# # df['n_incidents'] = df.groupby('user_id')['n_incidents'].cumsum()

# # # # # Para stat_cr != good_cr | stat_fe != good_fe acumulamos el numero de operaciones de tipo money_back
# # # df = df.sort_values(['created_at','created_at_fe'])

# # unique_cr = money_back & fee_accepted & (df['fee'] > 0 ) & ~df.duplicated(subset=['id_cr'], keep='first')
# # df['n_incidents_fees'] = unique_cr.groupby(df['user_id']).cumsum()

# # unique_cr = money_back & ((~df['stat_cr'].isin(good_cr)) | (~df['stat_fe'].isin(good_fe))) & ~df.duplicated(subset=['id_cr'], keep='first')
# # df['n_incidents_backs'] = unique_cr.groupby(df['user_id']).cumsum()


good_cr = ['approved', 'money_sent', 'pending', 'direct_debit_sent', 'active', 'money_back']
good_fe = ['confirmed', 'accepted', 'cr_regular']

# money_back = df['stat_cr'] == "money_back"
# fee_accepted = df['stat_fe'] == "accepted"

is_good_cr = df['stat_cr'].isin(good_cr)
is_good_fe = df['stat_fe'].isin(good_fe)
# bad_recovery_status_fe = ~df['recovery_status'].isin(["nice",'pending'])

# df = df.sort_values(['created_at','created_at_fe'])
df['n_incidents'] = ( (~df['stat_cr'].isin(good_cr)) | (~df['stat_fe'].isin(good_fe)) | (df['recovery_status'] != "nice")  ) & (df['amount'] > 0)
df['n_incidents'] = df.groupby('user_id')['n_incidents'].cumsum()

# df['n_incidents'] = ( (~is_good_cr) | (~is_good_fe) | (bad_recovery_status_fe)  ) & (df['amount'] > 0)
# df['n_incidents'] = df.groupby('user_id')['n_incidents'].cumsum()

# df['n_inc_back'] =  ~is_good_cr # & (df['amount'] > 0)
# df['n_inc_back'] = df.groupby('user_id')['n_inc_back'].cumsum()

# df['n_inc_fees'] = ( ~is_good_fe | bad_recovery_status_fe ) #& (df['amount'] > 0)
# df['n_inc_fees'] = df.groupby('user_id')['n_inc_fees'].cumsum()



# 34, 35 47 83 163 191 204 213
display(df[df['user_id'] == 47][fields].reset_index()) #47 tenemos un problema con n_fees
#display(df[df['n_backs'] <0][fields].reset_index().head(50)) #47 tenemos un problema con n_fees


,index,id_cr,user_id,created_at,moderated_at,to_reimbur,transfer_type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check_recov,n_recovery,n_inc_back,n_inc_fees,n_cr_fe_w,n_cr_fe_m,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,type,recovery_status,created_at_slot
0,0,3,47,2019-11-19,2019-11-20,16 days 09:02:06.488439,regular,canceled,1.0,0.0,0,0,1,0,1,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2019-12-05,16 days 09:02:06.488439,NaT,NaT,NaN,0,nice,13
1,53,118,47,2019-12-12,2019-12-12,6 days 23:59:59.999546,regular,rejected,100.0,0.0,0,0,1,0,2,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2019-12-19,6 days 23:59:59.999546,NaT,NaT,NaN,0,nice,14
2,1143,1218,47,2020-04-16,2020-04-16,14 days 00:00:00.577917,regular,money_back,100.0,0.0,0,1,0,0,2,0,1,1,cr_regular,0,NaT,NaT,NaN,2020-05-08,2020-04-30,14 days 00:00:00.577917,NaT,NaT,NaN,0,nice,8
3,1771,1832,47,2020-05-15,2020-05-15,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,1,1,1,cancelled,72,2020-05-29,2020-12-22,Postpone Cash Request 1832,2020-08-04,2020-09-17,125 days 05:55:49.550160,2020-07-19,2020-08-03,before,postpone,nice,16
4,1772,1832,47,2020-05-15,2020-05-15,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,2,1,1,cancelled,71,2020-05-29,2020-12-22,Postpone Cash Request 1832,2020-08-04,2020-09-17,125 days 05:55:49.550160,2020-06-19,2020-07-19,before,postpone,nice,16
5,1773,1832,47,2020-05-15,2020-05-15,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,3,1,1,cancelled,47,2020-06-01,2020-12-22,Postpone Cash Request 1832,2020-08-04,2020-09-17,125 days 05:55:49.550160,2020-08-03,2020-08-18,before,postpone,nice,16
6,1774,1832,47,2020-05-15,2020-05-15,125 days 05:55:49.550160,regular,money_back,10.0,5.0,0,2,1,0,2,4,1,1,cancelled,21,2020-06-23,2020-12-22,Postpone Cash Request 1832,2020-08-04,2020-09-17,125 days 05:55:49.550160,2020-06-05,2020-06-19,before,postpone,nice,16
7,1775,1832,47,2020-05-15,2020-05-15,125 days 05:55:49.550160,regular,money_back,10.0,5.0,1,2,0,0,2,4,1,1,accepted,1367,2020-07-14,2020-10-13,Postpone Cash Request 1832,2020-08-04,2020-09-17,125 days 05:55:49.550160,2020-08-18,2020-09-17,before,postpone,nice,16
8,11530,10942,47,2020-08-05,2020-08-05,55 days 12:25:43.181678,instant,money_back,10.0,5.0,2,3,0,0,2,4,2,2,accepted,2792,2020-08-05,2021-01-21,Instant Payment Cash Request 10942,2020-09-22,2020-09-29,55 days 12:25:43.181678,NaT,NaT,after,instant_payment,nice,9
9,11531,10942,47,2020-08-05,2020-08-05,55 days 12:25:43.181678,instant,money_back,10.0,5.0,3,3,0,0,2,4,2,2,accepted,5609,2020-09-01,2020-10-13,Postpone Cash Request 10942,2020-09-22,2020-09-29,55 days 12:25:43.181678,2020-09-04,2020-09-29,before,postpone,nice,9


In [9]:
# df = df.sort_values(['created_at','created_at_fe'])
# df['n_fees'] = (df['stat_cr'] == "money_back") & (df['stat_fe'] == "accepted") & (df['fee'] > 0)
# df['n_fees'] = df.groupby('user_id')['n_fees'].cumsum()

# # # Para stat_cr == "money_back" & stat_fe == "accepted" acumulamos el numero de operaciones de tipo money_back        
# df = df.sort_values(['created_at','created_at_fe'])
# unique_cr = (df['stat_cr'] == "money_back") & (df['amount'] > 0) & ~df.duplicated(subset=['id_cr'], keep='first')
# df['n_backs'] = unique_cr.groupby(df['user_id']).cumsum()-1
#df['created_at_slot'] = df['created_at'].dt.hour

#df['to_reimbur_d'] = df['to_reimbur'] / pd.to_timedelta(1, unit='d') #Min


#df['created_at'] = pd.to_datetime(df['created_at'])

# df = df.sort_values(['created_at','created_at_fe'])
# unique_cr = (df['stat_cr'] == "money_back") & (df['amount'] > 0) & ~df.duplicated(subset=['id_cr'], keep='first')
# df['n_backs'] = unique_cr.groupby(df['user_id']).cumsum() #-1
# #df['n_backs'] = (unique_cr.groupby(df['user_id']).cumsum() > 0).astype(int) + df['n_backs'].shift().fillna(0)
        
#user_frequencies = df.groupby('user_id').size().reset_index(name='n_user_cr_fe')
#df = pd.concat([df, user_frequencies], axis=0)    # concatenating along rows
#fields.append('n_user_cr_fe')
#display(user_frequencies)
#df['n_user_cr_fe'] = df.groupby('user_id').size().reset_index(name='n_user_cr_fe')


In [10]:
df = df_jo.copy()

# TODO: Reconsiderar poner de nuevo las fechas

df_hyper = df_jo[[
            'user_id',
            #'created_at', 
            'created_at_slot', 'created_at_dow',
            'amount',
            'needs_m_check_recov', 'n_fees', 'n_backs', 'n_recovery', 'n_inc_back','n_inc_fees',
            'transfer_type', 'charge_moment',
            #'reco_creation', 
            #'to_receive_ini', 'to_receive_bank', 'to_reimbur',
            #'to_reimbur_cash', 'to_end', 'to_send',
            #'send_at', 'paid_at',
            #'moderated_at', 
            'n_cr_fe_w', #'n_cr_fe_m', 
            'category',
            'inflation' , 'GBP_EUR', 'BTC_GBP', 'unemploy_rate',
            ]].copy()

# TODO: Separar para CR y FE 

# # Para CR recovery_status != "nice" acumulamos el numero de recovery_status que han tenido incidentes.        
df = df.sort_values(['created_at','created_at_fe'])
df['n_recovery'] = (df['recovery_status'] != "nice") & (df['amount'] > 0)
df['n_recovery'] = df.groupby('user_id')['n_recovery'].cumsum()


# # Para stat_cr != good_cr | stat_fe != good_fe acumulamos el numero de operaciones de tipo money_back
good_cr = ['approved', 'money_sent', 'pending', 'direct_debit_sent', 'active', 'money_back']
good_fe = ['confirmed', 'accepted', 'cr_regular']        
df = df.sort_values(['created_at','created_at_fe'])
df['n_incidents'] = ( (~df['stat_cr'].isin(good_cr)) | (~df['stat_fe'].isin(good_fe)) | (df['recovery_status'] != "nice")  ) & (df['amount'] > 0)
df['n_incidents'] = df.groupby('user_id')['n_incidents'].cumsum()



pm.format_to_dates(df, time_format='d') # 'min','s'
df = df.sort_values(by=['user_id', 'created_at'])
# 2024-12-11 TODO Alejandro Casos en los que poner n_backs -1 nos da un resultado incorrecto:
# 34, 35 47 83 163 191 204 213
display(df[df['user_id'] == 213][fields].reset_index()) #47 tenemos un problema con n_fees
#display(df[df['n_backs'] <0][fields].reset_index().head(50)) #47 tenemos un problema con n_fees


,index,id_cr,user_id,created_at,moderated_at,to_reimbur,transfer_type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check_recov,n_recovery,n_inc_back,n_inc_fees,n_cr_fe_w,n_cr_fe_m,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,type,recovery_status,created_at_slot
0,303,369,213,2020-01-03,2020-01-03,10 days 23:59:59.986271,regular,rejected,100.0,0.0,0,0,1,0,1,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2020-01-14,10 days 23:59:59.986271,NaT,NaT,NaN,0,nice,16
1,373,440,213,2020-01-13,2020-01-14,25 days 23:59:59.984713,regular,rejected,100.0,0.0,0,0,1,0,2,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2020-02-08,25 days 23:59:59.984713,NaT,NaT,NaN,0,nice,23
2,436,503,213,2020-01-25,2020-01-25,14 days 23:59:59.994118,regular,rejected,100.0,0.0,0,0,1,0,3,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2020-02-09,14 days 23:59:59.994118,NaT,NaT,NaN,0,nice,11
3,528,597,213,2020-02-03,2020-02-04,35 days 23:59:59.993716,regular,rejected,100.0,0.0,0,0,1,0,4,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2020-03-10,35 days 23:59:59.993716,NaT,NaT,NaN,0,nice,23
4,1370,1453,213,2020-04-29,2020-04-30,31 days 04:28:30.278620,regular,rejected,100.0,0.0,0,0,1,0,5,0,0,0,cr_regular,0,NaT,NaT,NaN,NaT,2020-05-30,31 days 04:28:30.278620,NaT,NaT,NaN,0,nice,17
5,5317,4997,213,2020-06-24,2020-06-24,34 days 06:22:44.180174,regular,money_back,100.0,5.0,1,1,1,1,5,1,3,3,accepted,5497,2020-08-30,2020-10-13,rejected direct debit,2021-01-15,2020-07-28,34 days 06:22:44.180174,NaT,NaT,after,incident,completed,15
6,5318,4997,213,2020-06-24,2020-06-24,34 days 06:22:44.180174,regular,money_back,100.0,5.0,2,1,1,2,5,2,3,3,accepted,9984,2020-09-30,2020-10-13,month delay on payment - 9/2020,2021-01-15,2020-07-28,34 days 06:22:44.180174,NaT,NaT,after,incident,completed,15
7,5319,4997,213,2020-06-24,2020-06-24,34 days 06:22:44.180174,regular,money_back,100.0,5.0,3,1,1,3,5,3,3,3,accepted,19930,2020-10-29,2020-10-29,month delay on payment - 9/2020,2021-01-15,2020-07-28,34 days 06:22:44.180174,NaT,NaT,after,incident,completed,15
8,10773,10342,213,2020-07-31,2020-08-01,27 days 03:30:35.656820,instant,money_back,100.0,5.0,3,2,1,4,5,4,3,3,cancelled,2515,2020-08-01,2020-12-15,Instant Payment Cash Request 10342,2021-01-15,2020-08-27,27 days 03:30:35.656820,NaT,NaT,after,instant_payment,completed,18
9,10774,10342,213,2020-07-31,2020-08-01,27 days 03:30:35.656820,instant,money_back,100.0,5.0,4,2,1,5,5,5,3,3,accepted,5438,2020-08-29,2020-10-13,rejected direct debit,2021-01-15,2020-08-27,27 days 03:30:35.656820,NaT,NaT,after,incident,completed,18
